In [1]:
from CA import *

In [15]:
cars, road = random_init_cars(100, 0.3)
pos = car_positions(road)
position_update(pos, cars, road)

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  2,
        0,  0,  0,  0,  0,  3,  0,  0,  0,  4,  0,  0,  0,  0,  5,  0,  0,
        0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  7,  0,  8,  0,  0,  0,  9,
        0,  0, 10,  0,  0,  0,  0,  0,  0,  0, 11,  0, 12,  0, 13,  0, 14,
        0,  0,  0,  0,  0,  0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       16,  0,  0,  0, 17,  0, 18,  0,  0,  0,  0,  0, 19,  0,  0])

In [ ]:
def experiment(iterations=1000, )